In [36]:
import re

from datasets import load_dataset

from soynlp.normalizer import repeat_normalize
import pandas as pd

## Load

In [31]:
data = pd.read_excel("/Users/mskim/Downloads/total.xlsx")
data = data.drop(columns="Unnamed: 0")
data = data.drop(data[data["text"].isnull()].index)

data

,id,genre,title,text
0,0,단편소설,단추,"그녀의 이름은 단추. 카페의 주인이다.\n'단추, 카페'에 필요한 것은 단추와 바늘..."
1,1,시,양을 찾아서,마침내 양은 사라졌다\n\n한 의식을 잃고서 나는 은주발에 담긴 눈*이 되고 싶었다...
2,2,동화,도깨비놀이터,"“정우야, 너 도깨비 보러 갈래?”\n\n태성이가 불쑥 그렇게 말했을 때 나는 조금..."
3,3,동시,엄마의 마음,나사못이\n\n나무를 뚫고\n\n들어갑니다\n\n한 바퀴\n\n두 바퀴\n\n빙글빙...
4,4,소설,선의 취향,미오가 선의 곁을 떠난 지 오늘로 100일째다. 누군가와 헤어진 지 100일째 되는...
...,...,...,...,...
21880,21880,시,형광등,자신의 귀한 몸 희생하며\n그대는 무엇을 하고 있는가\n몸이 뜨거워져 타들어 갈 때...
21881,21881,시,교실,이 곳은 전쟁터\n오직 1등급만이 살아남는\n가장 잔인한 전쟁터\n총칼대신 책과 연...
21882,21882,시,우주선,"톰, 잭, 피터 셋이 모였어.\n톰은 천은\n잭은 솜을\n피터는 바늘과 실을 가지고..."
21883,21883,시,토끼와 거북이,큰 산 아래\n짦은 다리 네개\n동그란 얼굴에\n꿈벅꿈벅 눈을 가진\n나는야 느릿느...


## Process

In [32]:
def preprocess(text):
    # 문제를 일으킬 수 있는 문자 제거
    bad_chars = {"\u200b": "", "…": " ... ", "\ufeff": ""}
    for bad_char in bad_chars:
        text = text.replace(bad_char, bad_chars[bad_char])

    error_chars = {"\u3000": " ", "\u2009": " ", "\u2002": " ", "\xa0": " "}
    for error_char in error_chars:
        text = text.replace(error_char, error_chars[error_char])

    # 이메일 제거
    text = re.sub(r"[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", "[이메일]", text).strip()

    # "#문자" 형식 어절 제거
    text = re.sub(r"#\S+", "", text).strip()

    # "@문자" 형식 어절 제거
    text = re.sub(r"@\w+", "", text).strip()

    # URL 제거
    text = re.sub(r"(http|https)?:\/\/\S+\b|www\.(\w+\.)+\S*", "[웹주소]", text).strip()
    text = re.sub(r"pic\.(\w+\.)+\S*", "[웹주소]", text).strip()

    # 뉴스 저작권 관련 텍스트 제거
    re_patterns = [
        r"\<저작권자(\(c\)|ⓒ|©|\(Copyright\)|(\(c\))|(\(C\))).+?\>",
        r"저작권자\(c\)|ⓒ|©|(Copyright)|(\(c\))|(\(C\))",
    ]

    for re_pattern in re_patterns:
        text = re.sub(re_pattern, "", text).strip()

    # 뉴스 내 포함된 이미지에 대한 레이블 제거
    text = re.sub(r"\(출처 ?= ?.+\) |\(사진 ?= ?.+\) |\(자료 ?= ?.+\)| \(자료사진\) |사진=.+기자 ", "", text).strip()

    # 중복 문자 처리
    text = repeat_normalize(text, num_repeats=2).strip()

    # 문제를 일으킬 수 있는 구두점 치환
    punct_mapping = {
        "‘": "'",
        "₹": "e",
        "´": "'",
        "°": "",
        "€": "e",
        "™": "tm",
        "√": " sqrt ",
        "×": "x",
        "²": "2",
        "—": "-",
        "–": "-",
        "’": "'",
        "_": "-",
        "`": "'",
        "“": '"',
        "”": '"',
        "“": '"',
        "£": "e",
        "∞": "infinity",
        "θ": "theta",
        "÷": "/",
        "α": "alpha",
        "•": ".",
        "à": "a",
        "−": "-",
        "β": "beta",
        "∅": "",
        "³": "3",
        "π": "pi",
        "*": "",
    }
    for p in punct_mapping:
        text = text.replace(p, punct_mapping[p])

    # 연속된 공백 치환
    text = re.sub(r"\s+", " ", text).strip()

    text = re.sub("\n{3,}", "\n\n", text)

    return text

In [34]:
d = data.to_dict()
for k, text in d["text"].items():
    d["text"][k] = preprocess(text)

data = pd.DataFrame(d)
data.to_csv("/Users/mskim/Downloads/total.csv")
data

,id,genre,title,text
0,0,단편소설,단추,"그녀의 이름은 단추. 카페의 주인이다. '단추, 카페'에 필요한 것은 단추와 바늘,..."
1,1,시,양을 찾아서,마침내 양은 사라졌다 한 의식을 잃고서 나는 은주발에 담긴 눈이 되고 싶었다 갈피가...
2,2,동화,도깨비놀이터,"""정우야, 너 도깨비 보러 갈래?"" 태성이가 불쑥 그렇게 말했을 때 나는 조금 얼떨..."
3,3,동시,엄마의 마음,나사못이 나무를 뚫고 들어갑니다 한 바퀴 두 바퀴 빙글빙글 돌다가 더는 돌 수 없어...
4,4,소설,선의 취향,미오가 선의 곁을 떠난 지 오늘로 100일째다. 누군가와 헤어진 지 100일째 되는...
...,...,...,...,...
21880,21880,시,형광등,자신의 귀한 몸 희생하며 그대는 무엇을 하고 있는가 몸이 뜨거워져 타들어 갈 때까지...
21881,21881,시,교실,이 곳은 전쟁터 오직 1등급만이 살아남는 가장 잔인한 전쟁터 총칼대신 책과 연필들고...
21882,21882,시,우주선,"톰, 잭, 피터 셋이 모였어. 톰은 천은 잭은 솜을 피터는 바늘과 실을 가지고. 그..."
21883,21883,시,토끼와 거북이,큰 산 아래 짦은 다리 네개 동그란 얼굴에 꿈벅꿈벅 눈을 가진 나는야 느릿느릿 거북...


## Collect specific genre

In [35]:
poem = data[data["genre"].apply(lambda x: "시" in x)]
poem = poem.reset_index().drop(columns="index")
poem.to_csv("/Users/mskim/Downloads/poem.csv")
poem

,id,genre,title,text
0,1,시,양을 찾아서,마침내 양은 사라졌다 한 의식을 잃고서 나는 은주발에 담긴 눈이 되고 싶었다 갈피가...
1,3,동시,엄마의 마음,나사못이 나무를 뚫고 들어갑니다 한 바퀴 두 바퀴 빙글빙글 돌다가 더는 돌 수 없어...
2,5,시,앵두나무 상영관,신호등은 봄을 켠다 길 하나 사이에 두고 마주 선 두 그루 이도시에 앵두가 없다는 ...
3,6,시조,금빛 질경이,"흙바람 길을 튼다, 길섶에 씨방 연다 비에 젖은 잎새 위에 숨 고르는 햇살 한 줌 ..."
4,10,시,폐가를 어루만지다,"""허물어지는 것은 새것을 위한 눈부신 산화 나는 철거될 농가의 마룻바닥에 가만 귀 ..."
...,...,...,...,...
21149,21880,시,형광등,자신의 귀한 몸 희생하며 그대는 무엇을 하고 있는가 몸이 뜨거워져 타들어 갈 때까지...
21150,21881,시,교실,이 곳은 전쟁터 오직 1등급만이 살아남는 가장 잔인한 전쟁터 총칼대신 책과 연필들고...
21151,21882,시,우주선,"톰, 잭, 피터 셋이 모였어. 톰은 천은 잭은 솜을 피터는 바늘과 실을 가지고. 그..."
21152,21883,시,토끼와 거북이,큰 산 아래 짦은 다리 네개 동그란 얼굴에 꿈벅꿈벅 눈을 가진 나는야 느릿느릿 거북...


## Split

In [ ]:
# dataset = load_dataset("csv", data_files="/Users/mskim/Downloads/poem.csv").remove_columns("Unnamed: 0")
# dataset = dataset["train"].train_test_split(test_size=0.1)
# dataset["validation"] = dataset["test"]
# dataset.pop("test")
# dataset

## Export

In [ ]:
# dataset.push_to_hub("total")